In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [3]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/Wilt%20Dataset.csv"
df = pd.read_csv(url, header = None)

X = df.drop(0, axis=1)
y = df[0]

y.value_counts()

0    4578
1     261
Name: 0, dtype: int64

In [4]:
#divide

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
#param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
#svm = SVC(kernel='rbf')

#grid_search = GridSearchCV(svm, param_grid, cv=5)
#grid_search.fit(X_train_scaled, y_train)

#best_params = grid_search.best_params_
#best_C = best_params['C']
#best_gamma = best_params['gamma']

best_C = 1
best_gamma = 1

#Fit the SVM with the best parameters:
svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)

In [8]:
svm.fit(X_train_scaled, y_train)

SVC(C=1, gamma=1)

In [9]:
# Make predictions on the test set
y_test_pred = svm.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", accuracy)

# Prediction rule is essentially: 1 if distance > 0
distances = svm.decision_function(X_test_scaled)
tmp = np.where(distances > 0, 1, 0)
print("Match between Prediction and Rule based on Distance")
accuracy_score(tmp, y_test_pred)

Test Accuracy: 0.9757231404958677
Match between Prediction and Rule based on Distance


1.0

In [10]:
from sklearn.calibration import CalibratedClassifierCV

In [11]:
svm_platt = CalibratedClassifierCV(svm)
svm_platt.fit(X_train_scaled, y_train)
y_test_prob = svm_platt.predict_proba(X_test_scaled)

In [12]:
y_test_pred = np.where(y_test_prob[:,1] > 0.5, 1, 0)
accuracy_score(y_test, y_test_pred)

0.9783057851239669

In [13]:
def svm_iterate_process(X, y):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    
    # Define the number of batches
    num_batches = 11
    
    # Randomly shuffle the data indices
    indices = np.random.permutation(len(X_train))
    
    # Calculate the batch size
    batch_size = len(X_train) // num_batches
    
    # Make predictions on the test set using majority voting
    preds_voting = np.zeros(len(y_test))
    # Make predictions on the test set using average of logit
    preds_distance = np.zeros(len(y_test))
    #Make predictions on the test set using average of probs
    preds_prob = np.zeros(len(y_test))

    
    # Split the training data into batches, fit a logistic regression model on each batch
    for i in range(num_batches):
        # Calculate the starting and ending indices for the current batch
        start_index = i * batch_size
        end_index = (i + 1) * batch_size
        
        # Select the current batch for training
        X_batch = X_train.iloc[indices[start_index:end_index]]
        y_batch = y_train.iloc[indices[start_index:end_index]]
        
        scaler = StandardScaler()
        X_batch_scaled = scaler.fit_transform(X_batch)
        X_test_scaled = scaler.transform(X_test)
        
        
        # Create a support vector machine model
        param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
        svm = SVC(kernel='rbf')

        grid_search = GridSearchCV(svm, param_grid, cv=5)
        grid_search.fit(X_batch_scaled, y_batch)

        best_params = grid_search.best_params_
        best_C = best_params['C']
        best_gamma = best_params['gamma']
        svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
        
        # Fit the model on the current batch
        svm.fit(X_batch_scaled, y_batch)
        
               
        # Accumulate the predictions using majority voting
        y_pred = svm.predict(X_test_scaled)
        preds_voting += (y_pred == 1)

    
        # Accumulate the predictions using majority voting
        y_pred = svm.decision_function(X_test_scaled)
        preds_distance += y_pred
        
        
        #Accumulate the probs
        svm_platt = CalibratedClassifierCV(svm)
        svm_platt.fit(X_batch_scaled, y_batch)
        y_pred = svm_platt.predict_proba(X_test_scaled)
        preds_prob += y_pred[:,1]
   
    
    accuracy = np.zeros(4)
    
    # Majority voting (selecting the most frequent prediction for each sample)
    final_predictions = np.where(preds_voting > num_batches / 2, 1, 0)
    accuracy[0] = accuracy_score(y_test, final_predictions)
    # Average of logit
    final_predictions = np.where(preds_distance > 0, 1, 0)
    accuracy[1] = accuracy_score(y_test, final_predictions)
    #Average of probs
    final_predictions = np.where(preds_prob / num_batches > 0.5, 1, 0)
    accuracy[2] = accuracy_score(y_test, final_predictions)
    
    # Train a model on all 11 batches of training data
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
    svm = SVC(kernel='rbf')

    grid_search = GridSearchCV(svm, param_grid, cv=5)
    grid_search.fit(X_train_scaled, y_train)

    best_params = grid_search.best_params_
    best_C = best_params['C']
    best_gamma = best_params['gamma']
    svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
    
    svm.fit(X_train_scaled, y_train)
    y_pred = svm.predict(X_test_scaled)
    accuracy[3] = accuracy_score(y_test, y_pred)
    
    return accuracy

In [14]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 4))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i] = svm_iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)

Accuracies: [[0.97365702 0.97417355 0.96900826 0.98192149]
 [0.98088843 0.97727273 0.97727273 0.98966942]
 [0.97365702 0.97365702 0.96539256 0.98347107]
 [0.96745868 0.96694215 0.96229339 0.98605372]
 [0.96952479 0.96952479 0.96280992 0.98295455]
 [0.97623967 0.9767562  0.96745868 0.98657025]
 [0.97727273 0.9731405  0.96952479 0.98553719]
 [0.97417355 0.97262397 0.96539256 0.98760331]
 [0.97623967 0.97572314 0.97469008 0.98863636]
 [0.97469008 0.9731405  0.96900826 0.98657025]
 [0.98192149 0.98243802 0.97933884 0.99173554]
 [0.97469008 0.97262397 0.96694215 0.98708678]
 [0.96590909 0.96952479 0.96177686 0.98140496]
 [0.97107438 0.97107438 0.96384298 0.98811983]
 [0.96745868 0.9643595  0.9607438  0.98450413]
 [0.97933884 0.97727273 0.97055785 0.98863636]
 [0.97520661 0.9731405  0.97262397 0.98708678]
 [0.96642562 0.96694215 0.9607438  0.98295455]
 [0.98295455 0.97882231 0.97159091 0.98657025]
 [0.97572314 0.97623967 0.96745868 0.98863636]
 [0.96642562 0.96332645 0.95867769 0.9803719 ]
 

In [19]:
np.mean(accuracies, axis = 0) 

array([0.97339876, 0.97251033, 0.96710744, 0.98566116])

In [20]:
np.std(accuracies, axis = 0)

array([0.00418261, 0.0037608 , 0.00454949, 0.0023182 ])

In [15]:
y_test_prob[:5,:]

array([[0.99256942, 0.00743058],
       [0.99253385, 0.00746615],
       [0.98578364, 0.01421636],
       [0.99656636, 0.00343364],
       [0.98924084, 0.01075916]])

In [16]:
from sklearn.linear_model import LogisticRegression
train_distances = svm.decision_function(X_train_scaled)
model = LogisticRegression(max_iter=500)
model.fit(train_distances.reshape(-1, 1), y_train)
tmp = model.predict_proba(distances.reshape(-1, 1))

In [17]:
tmp[:5,:]

array([[9.95072836e-01, 4.92716360e-03],
       [9.97095054e-01, 2.90494622e-03],
       [9.89613909e-01, 1.03860913e-02],
       [9.99040392e-01, 9.59607538e-04],
       [9.91580005e-01, 8.41999512e-03]])

In [18]:
tmp1 = np.where(tmp[:,1] > 0.5, 1, 0)
accuracy_score(y_test, tmp1)

0.9839876033057852